In [1]:
from nustar_pysolar import planning, io
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.time import TimeDelta
import warnings
warnings.filterwarnings('ignore')

# Download the list of occultation periods from the MOC at Berkeley.

## Note that the occultation periods typically only are stored at Berkeley for the *future* and not for the past. So this is only really useful for observation planning.

In [2]:
fname = io.download_occultation_times(outdir='../data/')
print(fname)

../data/NUSTAR.2021_115.SHADOW_ANALYSIS.txt


# Download the NuSTAR TLE archive.

This contains every two-line element (TLE) that we've received for the whole mission. We'll expand on how to use this later.

The `times`, `line1`, and `line2` elements are now the TLE elements for each epoch.

In [3]:
tlefile = io.download_tle(outdir='../data')
print(tlefile)
times, line1, line2 = io.read_tle_file(tlefile)

../data/NuSTAR.tle


# Here is where we define the observing window that we want to use.

Note that tstart and tend must be in the future otherwise you won't find any occultation times and sunlight_periods will return an error.

In [9]:
tstart = '2021-04-29T14:20:00'
tend = '2021-04-29T23:00:00'
orbits = planning.sunlight_periods(fname, tstart, tend)

In [10]:
orbits

[[<Time object: scale='utc' format='isot' value=2021-04-29T14:52:50.000>,
  <Time object: scale='utc' format='isot' value=2021-04-29T15:55:00.000>],
 [<Time object: scale='utc' format='isot' value=2021-04-29T16:29:30.000>,
  <Time object: scale='utc' format='isot' value=2021-04-29T17:31:40.000>],
 [<Time object: scale='utc' format='isot' value=2021-04-29T18:06:10.000>,
  <Time object: scale='utc' format='isot' value=2021-04-29T19:08:20.000>],
 [<Time object: scale='utc' format='isot' value=2021-04-29T19:42:50.000>,
  <Time object: scale='utc' format='isot' value=2021-04-29T20:45:00.000>],
 [<Time object: scale='utc' format='isot' value=2021-04-29T21:19:20.000>,
  <Time object: scale='utc' format='isot' value=2021-04-29T22:21:30.000>]]

In [11]:
# Get the solar parameter
from sunpy.coordinates import sun

angular_size = sun.angular_radius(t='now')
dx = angular_size.arcsec
print(dx)

953.091960344499


In [16]:
sun_pa = planning.get_nustar_roll(tstart, 0.)


pa = planning.get_nustar_roll(tstart, 45*u.deg)
print(tstart)
print("NuSTAR Roll angle for Det0 in NE quadrant: {}".format(pa))



2021-04-29T14:20:00
NuSTAR Roll angle for Det0 in NE quadrant: 20.663394935490903 deg


In [15]:

# Orbit 1 (AR)
offset = [0, 0.]*u.arcsec

ind = 1
orbit = orbits[0]
midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
sky_pos = planning.get_skyfield_position(midTime, offset, load_path='./data', parallax_correction=True)
print("Orbit: {}".format(ind))
print(f"Solar offset: {offset}")
print("Orbit start: {} Orbit end: {}".format(orbit[0].iso, orbit[1].iso))
print(f'Aim time: {midTime.iso} RA (deg): {sky_pos[0]:8.4f} Dec (deg): {sky_pos[1]:8.4f}')
print("")



# Orbit 1 (AR)
offset = [900, -300.]*u.arcsec

ind = 1
orbit = orbits[0]
midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
sky_pos = planning.get_skyfield_position(midTime, offset, load_path='./data', parallax_correction=True)
print("Orbit: {}".format(ind))
print(f"Solar offset: {offset}")
print("Orbit start: {} Orbit end: {}".format(orbit[0].iso, orbit[1].iso))
print(f'Aim time: {midTime.iso} RA (deg): {sky_pos[0]:8.4f} Dec (deg): {sky_pos[1]:8.4f}')
print("")


# Orbit 2a (AR)
offset = [900, -300.]*u.arcsec


ind = 2
orbit = orbits[1].copy()
print(orbit)
oldend = orbit[1]
newend = orbit[1] - TimeDelta(15*60., format = 'sec') # 15 minutes early
orbit[1] = newend

midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
sky_pos = planning.get_skyfield_position(midTime, offset, load_path='./data', parallax_correction=True)
print(f"Orbit: {ind}a")
print(f"Solar offset: {offset}")
print("Orbit start: {} Orbit end: {}".format(orbit[0].iso, orbit[1].iso))
print(f'Aim time: {midTime.iso} RA (deg): {sky_pos[0]:8.4f} Dec (deg): {sky_pos[1]:8.4f}')
print("")


# Orbit 2b (small thing)
offset = [900, -300.]*u.arcsec

orbit[0] = newend
orbit[1] = oldend
midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
sky_pos = planning.get_skyfield_position(midTime, offset, load_path='./data', parallax_correction=True)
print(f"Orbit: {ind}b")
print(f"Solar offset: {offset}")
print("Orbit start: {} Orbit end: {}".format(orbit[0].iso, orbit[1].iso))
print(f'Aim time: {midTime.iso} RA (deg): {sky_pos[0]:8.4f} Dec (deg): {sky_pos[1]:8.4f}')
print("")






# Orbit 3, 4 (PSP Footpoints)
offset = [900, -300.]*u.arcsec

for oi in [2, 3]:
    ind = oi+1

    orbit = orbits[oi]
    midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
    sky_pos = planning.get_skyfield_position(midTime, offset, load_path='./data', parallax_correction=True)
    print("Orbit: {}".format(ind))
    print(f"Solar offset: {offset}")
    print("Orbit start: {} Orbit end: {}".format(orbit[0].iso, orbit[1].iso))
    print(f'Aim time: {midTime.iso} RA (deg): {sky_pos[0]:8.4f} Dec (deg): {sky_pos[1]:8.4f}')
    print("")




Orbit: 1
Solar offset: [0. 0.] arcsec
Orbit start: 2021-04-29 14:52:50.000 Orbit end: 2021-04-29 15:55:00.000
Aim time: 2021-04-29 15:23:55.000 RA (deg):  36.8471 deg Dec (deg):  14.5729 deg

Orbit: 1
Solar offset: [ 900. -300.] arcsec
Orbit start: 2021-04-29 14:52:50.000 Orbit end: 2021-04-29 15:55:00.000
Aim time: 2021-04-29 15:23:55.000 RA (deg):  36.6472 deg Dec (deg):  14.3939 deg

[<Time object: scale='utc' format='isot' value=2021-04-29T16:29:30.000>, <Time object: scale='utc' format='isot' value=2021-04-29T17:31:40.000>]
Orbit: 2a
Solar offset: [ 900. -300.] arcsec
Orbit start: 2021-04-29 16:29:30.000 Orbit end: 2021-04-29 17:16:40.000
Aim time: 2021-04-29 16:53:05.000 RA (deg):  36.7074 deg Dec (deg):  14.4130 deg

Orbit: 2b
Solar offset: [ 900. -300.] arcsec
Orbit start: 2021-04-29 17:16:40.000 Orbit end: 2021-04-29 17:31:40.000
Aim time: 2021-04-29 17:24:10.000 RA (deg):  36.7239 deg Dec (deg):  14.4189 deg

Orbit: 3
Solar offset: [ 900. -300.] arcsec
Orbit start: 2021-04-29

In [47]:
test1 = SkyCoord(289.3792274160115,  -22.304595055979675, unit = 'deg')
orb1 = SkyCoord(289.3855, -22.3051, unit = 'deg')
orb1.separation(test1)

<Angle 0.00582518 deg>

In [10]:
orbit

[<Time object: scale='utc' format='isot' value=2021-01-14T11:10:00.000>,
 <Time object: scale='utc' format='isot' value=2021-01-14T12:12:50.000>]

In [38]:
import sunpy

In [39]:
sunpy.__version__

'2.0.1'

In [49]:
test1 = SkyCoord(289.898451566591, -22.158432904027155 , unit = 'deg')
orb1 = SkyCoord(289.9047,  -22.1589, unit = 'deg')
orb1.separation(test1)

<Angle 0.00580576 deg>

In [35]:
sun_pa = planning.get_nustar_roll(tstart, 0.)


pa = planning.get_nustar_roll(tstart, 45*u.deg)



offset = [0, 0.]*u.arcsec

ind = 1
orbit = orbits[0]
midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
sun_pos = planning.get_skyfield_position(midTime, offset, load_path='./data', parallax_correction=True)



# Orbit 1 (AR)
offset = [900, -300.]*u.arcsec

ind = 1
orbit = orbits[0]
midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
sky_pos = planning.get_skyfield_position(midTime, offset, load_path='./data', parallax_correction=True)

planning.make_test_region(sky_pos[0], sky_pos[1], pa, sun_pos[0], sun_pos[1], sun_pa)
print(pa)

20.663394935490903 deg


<Quantity [36.84713877, 14.57287177] deg>